In [34]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import psycopg2 as ps

In [35]:
# Importamos el csv con la información de las reservas, los hoteles y los clientes
import pandas as pd
data = pd.read_csv("../data/hoteles_unidos.csv", parse_dates=['fecha_reserva', 'inicio_estancia', 'final_estancia'], dtype={'id_cliente': str, 'id_hotel': str})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    15000 non-null  datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   nombre_hotel     15000 non-null  object        
 10  ciudad           15000 non-null  object        
 11  id_hotel         15000 non-null  object        
 12  estrellas        15000 non-null  float64       
 13  precio_medio     15000 non-null  float64       
dtypes: bool(1), datetime64[ns](3), float64

In [36]:
data[["id_hotel", "nombre_hotel"]]

,id_hotel,nombre_hotel
0,1,Hotel Monte Verde
1,2,Hotel Brisas del Mar
2,3,Hotel Camino del Sol
3,4,Hotel Puerta del Cielo
4,5,Hotel Encanto Real
...,...,...
14995,135,ibis Madrid Alcala de Henares La Garena
14996,128,ibis Madrid Alcobendas
14997,186,ibis budget Madrid Alcorcon Móstoles
14998,114,ibis Madrid Aeropuerto Barajas


In [38]:
data["precio_medio"]

0        276.612381
1        275.225921
2        269.998444
3        280.151243
4        278.243996
            ...    
14995     83.000000
14996     85.000000
14997     77.000000
14998    116.000000
14999    172.000000
Name: precio_medio, Length: 15000, dtype: float64

In [39]:
# Importamos el csv con los eventos de Madrid sacados de la API
eventos = pd.read_csv("../data/datos_eventos.csv", parse_dates=['fecha_inicio', 'fecha_fin'])
eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   nombre_evento      1259 non-null   object        
 1   url_evento         1259 non-null   object        
 2   codigo_postal      1131 non-null   float64       
 3   direccion          1131 non-null   object        
 4   horario            1027 non-null   object        
 5   organizacion       1149 non-null   object        
 6   fecha_inicio       1259 non-null   datetime64[ns]
 7   fecha_fin          1259 non-null   datetime64[ns]
 8   hora_fecha_inicio  1259 non-null   object        
 9   hora_fecha_fin     1259 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 98.5+ KB


In [40]:
# Nos conectamos a la bddd de postgres
conn = ps.connect(
    dbname="Proyecto_ETL",
    user="postgres",
    password="quique",
    host = "localhost",
    port = "5432")

In [41]:
# Creamos un cursor con el objeto conexion
cur = conn.cursor()

In [42]:
# Ejecutamos para saber si estamos conectados
cur.execute("SELECT version();")
cur.fetchone()

('PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit',)

In [43]:
# comprobamos que no hay valores duplicados
data.duplicated().sum()



np.int64(0)

In [ ]:
data_insert_ciudad = ["Madrid"]

In [ ]:
insert_query = """
INSERT INTO ciudad (nombre_ciudad) VALUES (%s)
"""

In [ ]:
conn.rollback()

In [ ]:
cur.execute(insert_query, data_insert_ciudad)
conn.commit()

In [ ]:
# vamos a comprobar que se ha creado correctamente
query_ciudad = """
    SELECT * 
    FROM ciudad; 
"""
cur.execute(query_ciudad)
cur.fetchall()

[(1, 'Madrid')]

In [ ]:
# Creamos la tabla eventos

conn.rollback()  # Rollback any previous failed transaction
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
ciudades = dict(cur.fetchall())
ciudades

{'Madrid': 1}

In [ ]:
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
ciudades = dict(cur.fetchall())
ciudades

{'Madrid': 1}

In [ ]:
data_to_insert_eventos = []

for i , row in eventos.iterrows():
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"]
    direccion = row["direccion"] if pd.notna(row["direccion"]) else None
    horario = row["horario"] if pd.notna(row["horario"]) else None
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"] if pd.notna(row["organizacion"]) else None
    id_ciudad = ciudades.get("Madrid")
    
data_to_insert_eventos.append((nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))
       
            

In [ ]:
len(data_to_insert_eventos)

In [ ]:
insert_query = """
INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""



In [ ]:
# Ejecutamos la query
cur.executemany(insert_query, data_to_insert_eventos)
conn.commit()

[(1,
  "Ça Marche 'Trabajos forzados'",
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=5c4302a2fa0e1910VgnVCM1000001d4a900aRCRD',
  28015,
  'CALLE CONDE DUQUE 9',
  '20:00',
  datetime.date(2025, 5, 7),
  datetime.date(2025, 5, 7),
  'Centro de Cultura Contemporánea Conde Duque',
  1)]

In [ ]:
# Comprobamos que se ha insertado correctamente
query_eventos = """
    SELECT * 
    FROM eventos; 
"""
cur.execute(query_eventos)
cur.fetchmany(4)

In [ ]:
# Insertamos la tabla hoteles

data_insert_hoteles = []

for _, row in data.iterrows():
    id_hotel = row["id_hotel"]
    nombre_hotel = row["nombre_hotel"] 
    competencia = row["competencia"] 
    valoracion = row["estrellas"] 
    id_ciudad = ciudades.get("Madrid") 

    hoteles = (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)

    if hoteles not in data_insert_hoteles:
        data_insert_hoteles.append(hoteles)

[('1', 'Hotel Monte Verde', False, 3.1036363636363635, 1),
 ('2', 'Hotel Brisas del Mar', False, 3.088014981273408, 1),
 ('3', 'Hotel Camino del Sol', False, 3.0675675675675675, 1),
 ('4', 'Hotel Puerta del Cielo', False, 3.02851711026616, 1),
 ('5', 'Hotel Encanto Real', False, 3.031311154598826, 1),
 ('6', 'Palacio del Sol', False, 2.992156862745098, 1),
 ('7', 'Hotel Jardines del Rey', False, 2.9362549800796813, 1),
 ('8', 'Hotel Las Estrellas', False, 2.906614785992218, 1),
 ('9', 'Gran Hotel Madrid', False, 3.0675675675675675, 1),
 ('10', 'Hotel Torre Dorada', False, 2.919921875, 1),
 ('11', 'Hotel Palacio Imperial', False, 3.0064794816414686, 1),
 ('12', 'Hotel Luz de Madrid', False, 3.0522388059701493, 1),
 ('13', 'Hotel Los Almendros', False, 3.0112570356472794, 1),
 ('14', 'Hotel Sol y Luna', False, 3.0089766606822264, 1),
 ('15', 'Hotel Mirador Real', False, 2.9774859287054407, 1),
 ('16', 'Hotel Rincón Sereno', False, 2.998046875, 1),
 ('17', 'Hotel Vista Alegre', False, 2.9

In [ ]:
data_insert_hoteles

In [ ]:
# Creamos la query de inserción de los datos de hotel
insert_query_hoteles = """ 
                        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s)
"""

In [44]:
# Creamos la tabla clientes

# revisar tipo de datos
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'nombre_hotel',
       'ciudad', 'id_hotel', 'estrellas', 'precio_medio'],
      dtype='object')

In [45]:
data_insert_clientes = []

for _, row in data.iterrows():

    id_cliente = row["id_cliente"]
    nombre = row["nombre"],
    apellido = row["apellido"],
    mail = row["mail"]

    tabla_cliente = (id_cliente, nombre, apellido, mail)

    if tabla_cliente not in data_insert_clientes:
        data_insert_clientes.append(tabla_cliente)

In [46]:
# Creamos la query de inserción de los datos de ciudad
insert_query_clientes = """ 
                        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
                        VALUES (%s, %s, %s, %s)
"""

In [47]:
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_clientes, data_insert_clientes)
conn.commit()

UniqueViolation: llave duplicada viola restricción de unicidad «clientes_pkey»
DETAIL:  Ya existe la llave (id_cliente)=(e13ac13a-2b7f-4fb6-a98f-5cd417859a23).


In [ ]:
cur.close() 
conn.close()

[(1, 'Hotel Monte Verde', False, 3.1036363636363635, 1),
 (2, 'Hotel Brisas del Mar', False, 3.088014981273408, 1),
 (3, 'Hotel Camino del Sol', False, 3.0675675675675675, 1)]